In [8]:
import pandas as pd

df = pd.read_csv("data_handling/rating.txt", sep=";")
df


,episode,Name,rating
0,1,Winter Is Coming,8.9
1,2,The Kingsroad,8.6
2,3,Lord Snow,8.5
3,4,"Cripples, Bastards, and Broken Things",8.6
4,5,The Wolf and the Lion,9.0
...,...,...,...
68,69,A Knight of the Seven Kingdoms,7.9
69,70,The Long Night,7.5
70,71,The Last of the Starks,5.5
71,72,The Bells,5.9
